# Energy Consumtion Netherlands

Energie is duur. Dat weet iedereen. Tijdens dit project gaan "de Potter Robin" & "Verhaegen Stijn" een analyse doen over het energie verbruik in Nederland.

## Inhoud analyse

1. `Lijst inhoud analyse`

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.rdd import RDD
import matplotlib.pyplot as plt
from pyspark.sql.types import *
from pyspark.sql.functions import col
import glob
import os

In [2]:
spark = SparkSession.builder.appName("EnergyConsumption").getOrCreate()
sc = spark.sparkContext

Path naar de data

In [3]:
path_gas = r"./data/Gas/"
path_electricity = r"./data/Electricity/"

In [88]:
def add_year(r, year):
    new_row = r + "," + year
    new_row = new_row.split(",")
    return new_row

De functie "make_rdd" gaat een spark RDD maken adhv een path naar de raw data. Tijdens het aanmaken van de RDD gaat men ook het jaar toevoegen van wanneer de data is opgenomen. Nadien worden alle headings eruit gefiltert.

In [102]:
def make_rdd(path):
    rdd_full = sc.emptyRDD()
    for filename in os.listdir(path):
        # Read file and assign data
        data = sc.textFile(path + filename)
        # Get year
        year = filename[-8:]
        year = year[0:4]
        # Create structure
        rdd = data.map(lambda r: add_year(r, year))
        # Add rdd to rdd_full
        rdd_full = rdd_full.union(rdd)
    heading = rdd_full.first()
    rdd_full = rdd_full.filter(lambda r: r != heading)
    return rdd_full

In [103]:
rdd_gas = make_rdd(path_gas)
rdd_gas.first()

['Coteq Netbeheer BV',
 'GAS Gastransport Services (GASUNIE)',
 'Margrietstraat',
 '4175GA',
 '7165BD',
 'HAAFTEN',
 '18',
 '100.0',
 '83.33',
 '78',
 'G4',
 '3457',
 '0.0',
 '0.0',
 '2013']

In [104]:
rdd_electricity = make_rdd(path_electricity)
rdd_electricity.first()

['Coteq Netbeheer BV',
 'Netbeheerder Centraal Overijssel B.V.',
 'Dorpsstraat',
 '7468CP',
 '7471AA',
 'ENTER',
 '19',
 '89.47',
 '94.74',
 '89',
 '1x35',
 '4122',
 '89.47',
 '0.0',
 '2013']

We gaan hier de RDD van electricity omzetten naar een DataFrame.

In [105]:
heading = ["net_manager","purchase_area","street","zipcode_from","zipcode_to","city","num_connections","delivery_perc","perc_of_active_connections","type_conn_perc","type_of_connection","annual_consume","annual_consume_lowtarif_perc","smartmeter_perc", "year"]
df_electricity = rdd_electricity.toDF(heading)

df_electricity.show(2, vertical = True)

-RECORD 0--------------------------------------------
 net_manager                  | Coteq Netbeheer BV   
 purchase_area                | Netbeheerder Cent... 
 street                       | Dorpsstraat          
 zipcode_from                 | 7468CP               
 zipcode_to                   | 7471AA               
 city                         | ENTER                
 num_connections              | 19                   
 delivery_perc                | 89.47                
 perc_of_active_connections   | 94.74                
 type_conn_perc               | 89                   
 type_of_connection           | 1x35                 
 annual_consume               | 4122                 
 annual_consume_lowtarif_perc | 89.47                
 smartmeter_perc              | 0.0                  
 year                         | 2013                 
-RECORD 1--------------------------------------------
 net_manager                  | Coteq Netbeheer BV   
 purchase_area              

En nu ook voor gas.

In [106]:
df_gas = rdd_gas.toDF(heading)

df_gas.show(2, vertical=True)

-RECORD 0--------------------------------------------
 net_manager                  | Coteq Netbeheer BV   
 purchase_area                | GAS Gastransport ... 
 street                       | Margrietstraat       
 zipcode_from                 | 4175GA               
 zipcode_to                   | 7165BD               
 city                         | HAAFTEN              
 num_connections              | 18                   
 delivery_perc                | 100.0                
 perc_of_active_connections   | 83.33                
 type_conn_perc               | 78                   
 type_of_connection           | G4                   
 annual_consume               | 3457                 
 annual_consume_lowtarif_perc | 0.0                  
 smartmeter_perc              | 0.0                  
 year                         | 2013                 
-RECORD 1--------------------------------------------
 net_manager                  | Coteq Netbeheer BV   
 purchase_area              

In [118]:
x_axis = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
y_axis = []

# annual_consume_year = plt.figure(figsize=(7, 7))
# ax = annual_consume_year.add_subplot(111)
# ax.bar(x_axis, y_axis, fc='darksalmon', align='center')
# plt.show()

In [120]:
df_electricity_production = spark.read.option("header","true").option("inferSchema", "true").csv("./data/Elektriciteit__aanbod_en_verbruik_12102021_102309.csv", sep=";")
electricity_production_raw = sc.textFile("./data/Elektriciteit__aanbod_en_verbruik_12102021_102309.csv")

df_electricity_production.show(vertical = True)
df_electricity_production.printSchema()

-RECORD 0---------------------------------------------------------------------------------
 Perioden                                                                        | 2013   
 Bruto productie (mln kWh)                                                       | 100875 
 Eigen verbruik bij elektr.productie (mln kWh)                                   | 3505   
 Netto productie/Netto productie, totaal (mln kWh)                               | 97370  
 Netto productie/Kernenergie (mln kWh)                                           | null   
 Netto productie/Brandstoffen/Brandstoffen, totaal (mln kWh)                     | null   
 Netto productie/Brandstoffen/Kolen (mln kWh)                                    | null   
 Netto productie/Brandstoffen/Olieproducten (mln kWh)                            | null   
 Netto productie/Brandstoffen/Aardgas (mln kWh)                                  | null   
 Netto productie/Brandstoffen/Biomassa (mln kWh)                                 | 5277   

In [ ]:
# Load a text file and convert each line to a Row, so that a DataFrame can be made.

electricity_production_parts = electricity_production_raw.map(lambda l: l.split(";"))
electricity_production_rows = electricity_production_parts.map(lambda l: Row(periode=l[0], netto_productie=l[3],
                                                                             netto_productie_brandstoffen=l[5],
                                                                             netto_productie_windenergie=l[12],
                                                                             netto_productie_zonnestroom=l[15]))

In [24]:
electricity_production_rows.take(2)


NameError: name 'electricity_production_rows' is not defined

In [ ]:

header = electricity_production_rows.first()
df_electricity_production = spark.createDataFrame(electricity_production_rows.filter(lambda row: row != header))

In [ ]:
df_electricity_production.show()

df2_electricity_production = df_electricity_production.selectExpr("cast(periode as string) periode",
                                                                  "cast(netto_productie as int) netto_productie",
                                                                  "cast(netto_productie_brandstoffen as int) netto_productie_brandstoffen",
                                                                  "cast(netto_productie_windenergie as int) netto_productie_windenergie",
                                                                  "cast(netto_productie_zonnestroom as int) netto_productie_zonnestroom")
df2_electricity_production.printSchema()

In [ ]:
df_pandas_electricity_production = df2_electricity_production.toPandas()
print(df_pandas_electricity_production)

In [ ]:
df_pandas_electricity_production.plot(x="periode", y="netto_productie", kind="bar")
